### ASP.NET COre HTTP Request Pipeline

The ASP.NET Core HTTP request pipeline is a series of middleware components that are invoked in a specific order. The pipeline is configured in the Startup class. The Startup class is where you configure the request pipeline and services that are used by your application. The request pipeline is configured by adding middleware components to an IApplicationBuilder instance that is provided by dependency injection.

-> Request -> Middleware -> Response


### Project Structure
E-Commerce API

- Token system to identify users
- Using Reddis for caching


#### Assignment
write a a middleware that would check if the request header has x-api-key

context.request.headers.containskey("x-api-key")
log header value

In [ ]:
using Ibs.Application.IbsMdm;
using Ibs.CommonHelpers;
using Ibs.Interfaces.IbsMdm;
using Ibs.Interfaces.IMoneytor;
using Ibs.Interfaces.wxSecure;
using Ibs.InvestmentAPI.Areas.Anchoria.Models;
using Ibs.InvestmentAPI.Helpers;
using Ibs.InvestmentAPI.Models;
using Ibs.InvestmentAPI.VfdBankApi;
using Ibs.NetCoreMailer;
using Ibs.Persistence.MdmEntities;
using Microsoft.AspNetCore.Authentication.JwtBearer;
using Microsoft.AspNetCore.Authorization;
using Microsoft.AspNetCore.Hosting;
using Microsoft.AspNetCore.Http;
using Microsoft.AspNetCore.Mvc;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.Logging;
using Newtonsoft.Json;
using System;
using System.Linq;
using System.Threading.Tasks;

namespace Ibs.InvestmentAPI.Areas.Anchoria.Controllers
{
    [ApiVersion("1.1")]
    [Route("api/v{v:apiversion}/mutual-fund")]
    [ApiController]
    [Authorize(AuthenticationSchemes = JwtBearerDefaults.AuthenticationScheme)]
    public class MutualFundSubController : ControllerBase
    {
        private readonly IPortfolioService _portfolioService;
        private readonly ILogger<MutualFundSubController> _logger;
        private readonly IMdmService _mdmService;
        private readonly IVfdBankSettings _vfdBankSettings;
        private readonly ISettingService _settingService;
        private readonly IWebHostEnvironment _env;
        private readonly IConfiguration _config;
        private readonly IMailSender _mail;
        public MutualFundSubController(IPortfolioService portfolioService, ILogger<MutualFundSubController> logger,
            IMdmService mdmService, IVfdBankSettings vfdBankSettings, ISettingService settingService, IWebHostEnvironment env, IConfiguration config, IMailSender mail)
        {
            _portfolioService = portfolioService;
            _logger = logger;
            _mdmService = mdmService;
            _vfdBankSettings = vfdBankSettings;
            _settingService = settingService;
            _env = env;
            _config = config;
            _mail = mail;
        }


        [Route("subscription")]
        [HttpPost]
        public async Task<IActionResult> Subscribe([FromBody] SubscribeModel model)
        {
            try
            {

                if (model == null) return StatusCode(StatusCodes.Status400BadRequest, new ResponseModel
                {
                    statusCode = StatusCodes.Status400BadRequest,
                    message = "Bad Request",
                    hasError = true
                });
                //_logger.LogInformation(JsonConvert.SerializeObject(model));

                if (model.client_unique_ref <= 0)
                    return StatusCode(StatusCodes.Status200OK, new ResponseModel
                    {
                        statusCode = StatusCodes.Status400BadRequest,
                        message = "Invalid client unique reference",
                        hasError = true
                    });

                if (string.IsNullOrEmpty(model.currency) || model.currency.Length > 3)
                    return StatusCode(StatusCodes.Status200OK, new ResponseModel
                    {
                        statusCode = StatusCodes.Status400BadRequest,
                        message = "Invalid currency",
                        hasError = true
                    });

                if (model.amount <= 0)
                    return StatusCode(StatusCodes.Status200OK, new ResponseModel
                    {
                        statusCode = StatusCodes.Status400BadRequest,
                        message = "Transaction amount can not be less than or equal to zero",
                        hasError = true
                    });

                if (model.product_id <= 0)
                    return StatusCode(StatusCodes.Status200OK, new ResponseModel
                    {
                        statusCode = StatusCodes.Status400BadRequest,
                        message = "Invalid product selected",
                        hasError = true
                    });

                var portfolio = _portfolioService.PortfolioById(model.product_id);

                if (!portfolio.Any()) return StatusCode(StatusCodes.Status200OK, new ResponseModel
                {
                    statusCode = StatusCodes.Status400BadRequest,
                    message = "Invalid product selected",
                    hasError = true
                });
                var currencyId = portfolio.First().IdCurrency;
                var clientCashBalance = await _portfolioService.FetchCustomerCashBalance(model.client_unique_ref, -1, currencyId, 0, DateTime.Now);



                if (clientCashBalance < model.amount) return StatusCode(StatusCodes.Status200OK, new ResponseModel
                {
                    statusCode = StatusCodes.Status400BadRequest,
                    message = "You do not have sufficient balance to carry out this transaction",
                    hasError = true
                });



                var products = await _portfolioService.FetchPublicPortfolios();

                var selectedProduct = products.Where(x => x.portfolioId == model.product_id).FirstOrDefault();
                if (selectedProduct == null)
                    return StatusCode(StatusCodes.Status200OK, new ResponseModel
                    {
                        statusCode = StatusCodes.Status400BadRequest,
                        message = "There is no product with the selected id",
                        hasError = true
                    });

                var isUnitBased = selectedProduct.ProductType.ToLower() == "MFUNDS".ToLower() ? true : false; //portfolio.FirstOrDefault().UnitBased.HasValue ? portfolio.FirstOrDefault().UnitBased.Value : false;



                if (!isUnitBased) return StatusCode(StatusCodes.Status200OK, new ResponseModel
                {
                    statusCode = StatusCodes.Status400BadRequest,
                    message = "There selected product is not a mutual fund product",
                    hasError = true
                });

                var currency = _portfolioService.FetchPortfolioCurrency(model.product_id);

                if (!model.currency.ToLower().Equals(currency.ToLower()))
                    return StatusCode(StatusCodes.Status200OK, new ResponseModel
                    {
                        statusCode = StatusCodes.Status400BadRequest,
                        message = $"Invalid product currency. The selected portfolio currency can only be {currency}",
                        hasError = true
                    });
                var minimumInvestmentAmount = selectedProduct.minSubAmount;
                if ((minimumInvestmentAmount ?? 0) > model.amount)
                {
                    return StatusCode(StatusCodes.Status200OK, new ResponseModel
                    {
                        statusCode = StatusCodes.Status400BadRequest,
                        message = $"The minimum amount for investment in the portfolio is {minimumInvestmentAmount}",
                        hasError = true
                    });
                }
                var IdPortfolioGroup = portfolio.FirstOrDefault().IdPortfolioGroup.HasValue ? portfolio.FirstOrDefault().IdPortfolioGroup.Value : 0;

                var portGroup = await _portfolioService.FetchPortfolioGroup(IdPortfolioGroup);

                if (portGroup == null) return StatusCode(StatusCodes.Status200OK, new ResponseModel
                {
                    statusCode = StatusCodes.Status400BadRequest,
                    message = "Portfolio group not set up",
                    hasError = true
                });

                if (string.IsNullOrWhiteSpace(portGroup.IdBankAccount01)) return StatusCode(StatusCodes.Status200OK, new ResponseModel
                {
                    statusCode = StatusCodes.Status400BadRequest,
                    message = "Portfolio lodgement account not set up",
                    hasError = true
                });


                var person = await _mdmService.FetchCustomerByUCID(model.client_unique_ref);

                if (person == null) return StatusCode(StatusCodes.Status200OK, new ResponseModel
                {
                    statusCode = StatusCodes.Status400BadRequest,
                    message = "You are not a known customer.",
                    hasError = true
                });


                var productBanks = await _portfolioService.FetchPortfolioBankAccounts();

                if (!productBanks.Any()) return StatusCode(StatusCodes.Status200OK, new ResponseModel
                {
                    statusCode = StatusCodes.Status400BadRequest,
                    message = "No product account number setup",
                    hasError = true
                });

                var productBank = productBanks.FirstOrDefault(x => x.portfolioId == model.product_id);

                if (productBank == null)
                    return StatusCode(StatusCodes.Status200OK, new ResponseModel
                    {
                        statusCode = StatusCodes.Status400BadRequest,
                        message = "The selected product does not have bank account",
                        hasError = true
                    });

                if (string.IsNullOrWhiteSpace(productBank.bankCode) || string.IsNullOrWhiteSpace(productBank.accountNumber))
                    return StatusCode(StatusCodes.Status200OK, new ResponseModel
                    {
                        statusCode = StatusCodes.Status400BadRequest,
                        message = "The selected product does not have bank account",
                        hasError = true
                    });

                var offerPrice = _portfolioService.GetOfferPriceAsAt(model.product_id, DateTime.Now.Date.AddDays(-1), model.amount);
                if (offerPrice == 0) return StatusCode(StatusCodes.Status200OK, new ResponseModel
                {
                    statusCode = StatusCodes.Status400BadRequest,
                    message = "Offer price has not been generated. Try again.",
                    hasError = true
                });

                if (model.goalId > 0)
                {
                    var goal = _mdmService.FetchAllUserGoals(model.client_unique_ref).Where(x => x.id == model.goalId).FirstOrDefault();
                    if (goal is null) return StatusCode(StatusCodes.Status200OK, new ResponseModel
                    {
                        statusCode = StatusCodes.Status400BadRequest,
                        message = "Goal doesn't exist.",
                        hasError = true
                    });
                }
                var isLive = _config.GetSection("APIDev:InProduction").Value != null ? bool.Parse(_config.GetSection("APIDev:InProduction").Value) : false;
                var approvalResponse = false;
                var portfolioContributorId = await _portfolioService.OpenCustomerCashAccount(ucid: model.client_unique_ref, currencyId: currencyId, model.product_id, true);
                var transferResponse = false;
                var transactionDate = DateTime.Now;

                if (portfolioContributorId > 0)
                {
                    var IdPortfolioContributorAccount = await _portfolioService.AddCustomerTransaction(model.product_id, portfolioContributorId,
                   transactionDate, transactionDate, model.amount, "S", $"{person.lastName} {person.Othername} [{model.product_id}]", model.currency, $"Subcription via AISL App by {person.lastName} on {DateTime.Now} ================================", $"{person.firstName}", true, "API");

                    var addDebitLeg = _portfolioService.AddCustomerDebitTransaction(model.client_unique_ref, DateTime.Now, DateTime.Now, model.amount, "S", "Mutual Fund Subscription", "NGN", "Mutual Fund subscription", person.lastName);                   

                    if (approvalResponse)
                    {
                        if (isLive)
                        {
                            transferResponse = await TransferFundToMutualFundAccount(productBank.accountNumber, productBank.bankCode, model.amount.ToString(), model.product_id, model.client_unique_ref, person);
                            VfdBankAPIErrorLog webHookModel = new()
                            {
                                ErrorDate = DateTime.Now,
                                APIEndPoint = "Transfer fund to mutual fund account",
                                ErrorCode = model.client_unique_ref.ToString(),
                                ErrorMessage = $"Outward_Account Number:{productBank.accountNumber}_Bank Code:{productBank.bankCode}_Amount:{model.amount}",
                            };

                            await _mdmService.LogVfdBankError(webHookModel);
                            var anchoriaWebHookModel = new AnchoriaWebHookNoticeModel
                            {
                                creditAmount = model.amount.ToString(),
                                currentBalance = clientCashBalance.ToString(),
                                custReferenceNumber = model.client_unique_ref.ToString(),
                                timestamp = DateHelper.timeStamp,
                                transactionType = AnchoriaWebHookTransType.MUTUAL_FUND_SUB,
                                transactionDetails = $"MUT FUND SUB: {person.firstName} {person.lastName}",
                                senderName = $"{person.firstName} {person.lastName}"

                            };
                            if (transferResponse)
                            {
                                var notificationUrl = _config.GetSection("APIDev:WebhookNotificationUrl").Value != null ? _config.GetSection("APIDev:WebhookNotificationUrl").Value : null;
                                AnchoriaWebHookApiExtensions.PostMessage(anchoriaWebHookModel, notificationUrl);
                                approvalResponse = await _portfolioService.MutualFundBuySellApproval(model.product_id, "S", person.firstName,
                       IdPortfolioContributorAccount, portGroup.IdBankAccount01, DateTime.Now);
                            }
                            else
                            {
                                return StatusCode(StatusCodes.Status500InternalServerError, new ResponseModel
                                {
                                    statusCode = StatusCodes.Status400BadRequest,
                                    message = "Transaction Failed",
                                    hasError = true
                                });
                            }

                        }
                        return StatusCode(StatusCodes.Status200OK, new ResponseModel
                        {
                            statusCode = StatusCodes.Status400BadRequest,
                            message = "Mutual fund subscription was completed successfully",
                            hasError = false
                        });
                    }
                    else
                    {
                        return StatusCode(StatusCodes.Status500InternalServerError, new ResponseModel
                        {
                            statusCode = StatusCodes.Status400BadRequest,
                            message = "Transaction Failed",
                            hasError = true
                        });
                    }

                }
                else
                {
                    return StatusCode(StatusCodes.Status500InternalServerError, new ResponseModel
                    {
                        statusCode = StatusCodes.Status500InternalServerError,
                        message = "Internal Server Error",
                        hasError = true
                    });
                }

            }
            catch (Exception ex)
            {
                _logger.LogError(ex, "Mutual Fund Subscription :: AISL APP (API)");

                return StatusCode(StatusCodes.Status500InternalServerError, new ResponseModel
                {
                    statusCode = StatusCodes.Status500InternalServerError,
                    message = " Internal Server Error",
                    hasError = false
                });
            }
        
        }



        [ApiExplorerSettings(IgnoreApi = true)]
        private async Task<bool> TransferFundToMutualFundAccount(string accountNumber, string bankCode, string amount, int productId, int client_unique_ref, PersonBasicInfoViewModel person)
        {
            var transferType = bankCode == "999999" ? TransferType.VFD_BANK : TransferType.OTHER_BANK;
            var recipientModel = new TransferModel() { accountNo = accountNumber, bank = bankCode, transfer_type = transferType };
            var vfdBankApi = new VfdBankAPI(_vfdBankSettings);
            var debitSuccessful = false;
            var recipient = await vfdBankApi.FetchRecepient(recipientModel);
            if (recipient.status == "00")
            {
                var poolAccountDetails = await vfdBankApi.AccountEnquiry();

                if (poolAccountDetails.status == "00")
                {
                    var from = poolAccountDetails.data;

                    var transModel = new TransferDetailsModel()
                    {
                        ToAccount = recipient.data.account.number,
                        ToBank = recipientModel.bank,
                        Amount = amount,
                        ToBvn = recipient.data.bvn,
                        Remark = $"{person.firstName} {person.lastName} {productId}".ToUpper(),
                        ToKyc = recipient.data.status,
                        ToSession = recipient.data.account.id,
                        ToClient = recipient.data.name,
                        ToSavingsId = string.IsNullOrWhiteSpace(recipient.data.account.id) ? "" : recipient.data.account.id,
                        TransferType = transferType,
                        Reference = $"AAM_{transferType}_{Guid.NewGuid().ToString()}",
                        Signature = string.Empty,
                        FromAccount = from.accountNo,
                        FromBvn = "",
                        FromClient = from.client,
                        FromClientId = from.clientId,
                        FromSavingsId = from.accountId,
                        ToClientId = string.IsNullOrWhiteSpace(recipient.data.clientId) ? "" : recipient.data.clientId
                    };

                    transModel.Signature = $"{transModel.FromAccount}{transModel.ToAccount}".EncryptSha512();

                    var transferResponse = await vfdBankApi.WalletTransferAsync(transModel);

                    //Log outward transfer success log

                    await _mdmService.LogOutwardTransfer(new Persistence.MdmEntities.VfdBankOutwardTransferLog
                    {
                        Amount = Convert.ToDecimal(transModel.Amount),
                        FromAccount = transModel.FromAccount,
                        FromClient = transModel.FromClient,
                        ToAccount = transModel.ToAccount,
                        ToBank = transModel.ToBank,
                        ToClient = transModel.ToClient,
                        TransactionDate = DateTime.Now,
                        TransRef = transferResponse.data.txnId

                    });

                    WebHookModel webHookModel = new()
                    {
                        loggedDate = DateTime.Now,
                        amount = Convert.ToDecimal(amount),
                        originAccount = transModel.FromAccount,
                        originAccountName = transModel.ToAccount,
                        originBank = transModel.ToBank,
                        originNarration = $"POOL TRANSACTION_{transferResponse.status}",
                        rawPayload = transModel.ToString(),
                        referenceNo = transModel.Reference,
                        walletAccountNo = transModel.ToAccount,
                        transactionTimeStamp = DateTime.Now.ToString()
                    };

                    await _mdmService.LogWebHookRequest(webHookModel);
                    if (transferResponse.status == "00")
                    {
                        debitSuccessful = true;
                        //return StatusCode(StatusCodes.Status200OK, new ResponseModel
                        //{
                        //    statusCode = StatusCodes.Status200OK,
                        //    message = "Mutual fund subscription was completed successfully",
                        //    hasError = false
                        //});
                    }
                    else
                    {


                        await _mdmService.LogVfdBankError(new Persistence.MdmEntities.VfdBankAPIErrorLog
                        {
                            APIEndPoint = "wallet2/transfer?source=pool",
                            ErrorCode = transferResponse.status,
                            ErrorDate = DateTime.Now,
                            ErrorMessage = transferResponse.message
                        });
                        debitSuccessful = false;

                        //return StatusCode(StatusCodes.Status200OK, new ResponseModel
                        //{
                        //    statusCode = StatusCodes.Status200OK,
                        //    message = transferResponse.message,
                        //    hasError = true
                        //});
                    }

                }
                else
                {

                    await _mdmService.LogVfdBankError(new Persistence.MdmEntities.VfdBankAPIErrorLog
                    {
                        APIEndPoint = "wallet2/account/enquiry",
                        ErrorCode = poolAccountDetails.status,
                        ErrorDate = DateTime.Now,
                        ErrorMessage = poolAccountDetails.message
                    });
                    debitSuccessful = false;
                    //return StatusCode(StatusCodes.Status200OK, new ResponseModel
                    //{
                    //    statusCode = StatusCodes.Status200OK,
                    //    message = poolAccountDetails.message,
                    //    hasError = true
                    //});
                }

            }
            else
            {


                await _mdmService.LogVfdBankError(new Persistence.MdmEntities.VfdBankAPIErrorLog
                {
                    APIEndPoint = "wallet2/transfer/recipient",
                    ErrorCode = recipient.status,
                    ErrorDate = DateTime.Now,
                    ErrorMessage = recipient.message
                });
                debitSuccessful = false;
                //return StatusCode(StatusCodes.Status200OK, new ResponseModel
                //{
                //    statusCode = StatusCodes.Status200OK,
                //    message = recipient.message,
                //    hasError = true
                //});
            }
            return debitSuccessful;


        }



        [Route("balance")]
        [HttpPost]
        public async Task<IActionResult> ClientMutualFundbalance([FromBody] SubscriptionEnqModel model)
        {
            try
            {
                if (model == null) return StatusCode(StatusCodes.Status400BadRequest, new ResponseModel
                {
                    statusCode = StatusCodes.Status400BadRequest,
                    message = "Bad Request",
                    hasError = true
                });

                var currCustomer = _portfolioService.FetchClientTransactionsByUCIDAndFundId(model.client_unique_ref, model.product_id).FirstOrDefault();

                if (currCustomer == null) return StatusCode(StatusCodes.Status200OK, new ResponseModel
                {
                    statusCode = StatusCodes.Status200OK,
                    message = "",
                    hasError = false,
                    data = new
                    {
                        principal = "0.00",
                        interest = "0.00",
                        total = "0.00"
                    }
                });


                var portInterestModel = await _portfolioService.GetPortfolioInterest(model.product_id, currCustomer.id_portcontributor, DateTime.Now);
                string specifier = "N";
                return StatusCode(StatusCodes.Status200OK, new ResponseModel
                {
                    statusCode = StatusCodes.Status200OK,
                    message = "",
                    hasError = false,
                    data = new
                    {
                        principal = portInterestModel.principal.ToString(specifier),
                        interest = portInterestModel.interest.ToString(specifier),
                        total = portInterestModel.total.ToString(specifier)
                    }
                });
            }
            catch (Exception ex)
            {
                _logger.LogError(ex, "Mutual Fund Balance Fetch Details :: AISL APP (API)");

                return StatusCode(StatusCodes.Status500InternalServerError, new ResponseModel
                {
                    statusCode = StatusCodes.Status500InternalServerError,
                    message = " Internal Server Error",
                    hasError = false
                });
            }
        }


        [Route("redemption/details")]
        [HttpPost]
        public async Task<IActionResult> FetchRedemptionDetails([FromBody] SubscriptionEnqModel model)
        {
            try
            {
                if (model == null) return StatusCode(StatusCodes.Status400BadRequest);



                if (model.amount < 0 || model.amount == 0)
                {
                    return StatusCode(StatusCodes.Status200OK, new ResponseModel
                    {
                        statusCode = StatusCodes.Status200OK,
                        message = "Amount to redeem cannot be less than or equal to zero",
                        hasError = true
                    });
                }

                if (model.amount > model.balance)
                {

                    return StatusCode(StatusCodes.Status200OK, new ResponseModel
                    {
                        statusCode = StatusCodes.Status200OK,
                        message = "Amount to redeem cannot be greater than available balance",
                        hasError = true
                    });
                }

                var amountToRedeem = model.amount;

                var currCustomer = _portfolioService.FetchClientTransactionsByUCIDAndFundId(model.client_unique_ref, model.product_id).FirstOrDefault();

                if (currCustomer == null) return StatusCode(StatusCodes.Status200OK, new ResponseModel
                {
                    statusCode = StatusCodes.Status200OK,
                    message = "Invalid customer",
                    hasError = true
                });


                var valuationDate = await _portfolioService.GetLastValuationDate(model.product_id);
                var salesDate = DateTime.Now;
                //var yesterDay = salesDate.Date.AddDays(-1);

                //if(valuationDate.Date < yesterDay)
                //{
                //    return StatusCode(StatusCodes.Status200OK, new ResponseModel
                //    {
                //        statusCode = StatusCodes.Status200OK,
                //        message = $"Valuation for {salesDate.Date.ToString("dd MMM yyyy")} not found. Please try again later",
                //        hasError = true
                //    });
                //}

                var bidPrice = await _portfolioService.GetBidPriceAsAt(model.product_id, DateTime.Now.Date.AddDays(-1), 0);
                if (bidPrice == 0) return StatusCode(StatusCodes.Status200OK, new ResponseModel
                {
                    statusCode = StatusCodes.Status200OK,
                    message = "Offer price has not been generated. Try again.",
                    hasError = true
                });


                var totalAmount = amountToRedeem / Convert.ToDecimal(bidPrice);


                _portfolioService.DeleteTempRedemptionRequest(model.product_id, currCustomer.id_portcontributor);


                var redemptionList = await _portfolioService.AutoRedemption(currCustomer.id_portcontributor, model.product_id,
                    salesDate.Date, totalAmount, true, false);


                if (redemptionList.Any())
                {
                    _portfolioService.InsertTempRedemptionRequest(redemptionList);
                }

                var netSettlementAmount = redemptionList.Sum(x => x.NetSettlement);
                var totalNoOfUnits = redemptionList.Sum(x => x.NoOfUnits);

                return StatusCode(StatusCodes.Status200OK, new ResponseModel
                {
                    statusCode = StatusCodes.Status200OK,
                    hasError = false,
                    data = new
                    {
                        certList = redemptionList,
                        noOfUnit = totalNoOfUnits.ToString("#,##.00"),
                        netSettlement = netSettlementAmount.ToString("#,##.00")
                    }
                });

            }
            catch (Exception ex)
            {
                _logger.LogError(ex, "Mutual Fund Redemption Fetch Details :: AISL APP (API)");

                return StatusCode(StatusCodes.Status500InternalServerError, new ResponseModel
                {
                    statusCode = StatusCodes.Status500InternalServerError,
                    message = " Internal Server Error",
                    hasError = false
                });
            }
        }



        [Route("redemption/confirm")]
        [HttpPost]
        public async Task<IActionResult> ProcessRedemption([FromBody] SubscriptionEnqModel model)
        {
            try
            {

                if (model == null) return StatusCode(StatusCodes.Status400BadRequest, new ResponseModel
                {
                    statusCode = StatusCodes.Status400BadRequest,
                    message = "Bad Request",
                    hasError = true
                });

                var currCustomer = _portfolioService.FetchClientTransactionsByUCIDAndFundId(model.client_unique_ref, model.product_id).FirstOrDefault();

                if (currCustomer == null) return StatusCode(StatusCodes.Status200OK, new ResponseModel
                {
                    statusCode = StatusCodes.Status200OK,
                    message = "Invalid customer",
                    hasError = true
                });

                var customerOldTransactions = _portfolioService.FetchClientTransactions(model.client_unique_ref, model.product_id);

                if (customerOldTransactions.Any(x => x.transactionType == "R" && (x.status == "P" || x.status == "T" || x.status == "B" || x.status == "V")))
                {
                    return StatusCode(StatusCodes.Status200OK, new ResponseModel
                    {
                        statusCode = StatusCodes.Status200OK,
                        message = "Please note that you have a pending redemption request. Please try again later",
                        hasError = true
                    });
                }

                var response = await _portfolioService.AddRedemptionRequest(model.product_id, currCustomer.id_portcontributor);
                var product = await _portfolioService.FetchOnlinePortfolio(model.product_id);
                var person = await _mdmService.FetchCustomerByUCID(model.client_unique_ref);

                var redemptionEmail = _settingService.GetSetting("REDEMPTION_NOTIFICATION_EMAIL");
                var customerCarePhone = _settingService.GetSetting("CustomerCareLines");

                var templateBuilder = new APIHelpers(_env);
                var customerFullName = $"{person.firstName} {person.lastName}".ToUpper();
                var inHouseMailBody = templateBuilder.BuildNewRedeemptionNotificationTemplate("Anchoria Asset Management Ltd",
                   response.ToString("#,##.00"), product.Description, currCustomer.portfolioName, customerFullName);
                await _mail.SendMail("Redemption Request", redemptionEmail, inHouseMailBody);


                return StatusCode(StatusCodes.Status200OK, new ResponseModel
                {
                    statusCode = StatusCodes.Status200OK,
                    message = "Your redemption request was successful",
                    hasError = false
                });
            }
            catch (Exception ex)
            {
                _logger.LogError(ex, "Mutual Fund Redemption Fetch Details :: AISL APP (API)");

                return StatusCode(StatusCodes.Status500InternalServerError, new ResponseModel
                {
                    statusCode = StatusCodes.Status500InternalServerError,
                    message = " Internal Server Error",
                    hasError = false
                });
            }
        }

    }
}
